<a href="https://colab.research.google.com/github/gbargaje/Deep-Learning/blob/master/ANN_from_scratch_IRIS_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder

iris = load_iris()
X = iris['data']
Y = iris.target

one_hot_encoder = OneHotEncoder(sparse=False)
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [0]:
from sklearn.model_selection import train_test_split
#spliting of data into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
#spliting of training data into training and validation data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [0]:
def InitializeWeights(nodes):
    #Initialize weights with random values in [-1, 1] (including bias)
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights
def Train(X, Y, lr, weights):
    layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        
        activations = ForwardPropagation(x, weights, layers)
        weights = BackPropagation(y, activations, weights, layers)

    return weights

In [0]:
def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    
    for j in range(layers):
        activation = Sigmoid(np.dot(layer_input, weights[j].T))
        activations.append(activation)
        layer_input = np.append(1, activation) # Augment with bias
    return activations
  
def Sigmoid(x):
    return (2 / (1 + np.exp(-2*x)))-1
  
def SigmoidDerivative(x):
    return 1-np.multiply(x, x)

In [0]:
def BackPropagation(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1):
        currActivation = activations[j]
        
        if(j > 1):
            # Augment previous activation
            prevActivation = np.append(1, activations[j-1])
        else:
            # First hidden layer, prevActivation is input (without bias)
            prevActivation = activations[0]
        
        delta = np.multiply(error, SigmoidDerivative(currActivation))
        weights[j-1] += lr * np.multiply(delta.T, prevActivation)

        w = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, w) # Calculate error for current layer
    
    return weights

In [0]:

def NeuralNetwork(X_train, Y_train, X_val=None, Y_val=None, epochs=10, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)

        if(epoch % 50 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
            if X_val.any():
                print("Validation Accuracy:{}".format(Accuracy(X_val, Y_val, weights)))
            
    return weights

In [0]:
def Predict(item, weights):
    layers = len(weights)
    item = np.append(1, item) # Augment feature vector
    
    ##_Forward Propagation_##
    activations = ForwardPropagation(item, weights, layers)
    
    outputFinal = activations[-1].A1
    index = FindMaxActivation(outputFinal)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # Set guessed class to 1

    return y # Return prediction vector


def FindMaxActivation(output):
    """Find max activation in output"""
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    return index

In [0]:
def Accuracy(X, Y, weights):
    correct = 0
    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = Predict(x, weights)

        if(y == guess):
            # Guessed correctly
            correct += 1
    return correct / len(X)


In [23]:
#print(len(X[0]))
#print(len(Y[0]))
layers = [f, 10, 5, o] # Number of nodes in layers
lr, epochs = 0.1, 1000

weights = NeuralNetwork(X_train, Y_train, X_val, Y_val, epochs=epochs, nodes=layers, lr=lr)

print("Testing Accuracy: {}".format(Accuracy(X_test, Y_test, weights)))


4
3
Epoch 50
Training Accuracy:0.9824561403508771
Validation Accuracy:0.9230769230769231
Epoch 100
Training Accuracy:0.9385964912280702
Validation Accuracy:0.7692307692307693
Epoch 150
Training Accuracy:0.9385964912280702
Validation Accuracy:0.7692307692307693
Epoch 200
Training Accuracy:0.9385964912280702
Validation Accuracy:0.8461538461538461
Epoch 250
Training Accuracy:0.9736842105263158
Validation Accuracy:0.8461538461538461
Epoch 300
Training Accuracy:0.9122807017543859
Validation Accuracy:0.7692307692307693
Epoch 350
Training Accuracy:0.9736842105263158
Validation Accuracy:0.8461538461538461
Epoch 400
Training Accuracy:0.9385964912280702
Validation Accuracy:0.7692307692307693
Epoch 450
Training Accuracy:0.9385964912280702
Validation Accuracy:0.7692307692307693
Epoch 500
Training Accuracy:0.9736842105263158
Validation Accuracy:0.8461538461538461
Epoch 550
Training Accuracy:0.9649122807017544
Validation Accuracy:0.8461538461538461
Epoch 600
Training Accuracy:0.9298245614035088
Vali